# Example: Properties

Adapted from: https://kthpanor.github.io/echem/docs/tutorials/vib_ir_calc.html

#### Imports

In [ ]:
from pathlib import Path

import numpy as np
import torch
import dxtb
import py3Dmol as p3d
from tad_mctc.io import read
from tad_mctc.units import AU2AA
from tad_mctc.convert import number_to_symbol, tensor_to_numpy

#### Functions

In [ ]:
def get_normal_mode(numbers, positions, normal_mode):
    elements = number_to_symbol(numbers)
    numbers = tensor_to_numpy(numbers)
    positions = tensor_to_numpy(positions * AU2AA)
    natm = numbers.shape[-1]

    vib_xyz = "%d\n\n" % natm
    nm = normal_mode.reshape(natm, 3)
    for i in range(natm):
        # add coordinates:
        vib_xyz += elements[i] + " %15.7f %15.7f %15.7f " % (
            positions[i, 0],
            positions[i, 1],
            positions[i, 2],
        )
        # add displacements:
        vib_xyz += "%15.7f %15.7f %15.7f\n" % (nm[i, 0], nm[i, 1], nm[i, 2])
    return vib_xyz

#### dxtb calculation

In [ ]:
device = torch.device("cpu")
dd: dxtb.typing.DD = {"device": device, "dtype": torch.double}

f = Path(globals()["_dh"][0]) / "molecules" / "h2o.coord"
numbers, positions = read.read_from_path(f, **dd, ftype="tm")
charge = torch.tensor(0.0, **dd)

opts = {
    "scf_mode": "full",
    "mixer": "anderson",
    "verbosity": 6,
    "f_atol": 1e-6,
    "x_atol": 1e-6,
}

calc = dxtb.Calculator(numbers, dxtb.GFN1_XTB, opts=opts, **dd)

In [ ]:
num_vibres = calc.vibration_numerical(positions, charge)
num_vibres.use_common_units()

In [ ]:
vibres = calc.vibration(positions.clone().requires_grad_(True), charge)
vibres.use_common_units()

In [ ]:
print("Shape of numbers", numbers.shape)
print("Shape of modes", vibres.modes.shape)
print("Shape of freqs", vibres.freqs.shape)

In [ ]:
modes = vibres.modes / torch.norm(vibres.modes, dim=-2, keepdim=True)
nummodes = num_vibres.modes / torch.norm(num_vibres.modes, dim=-2, keepdim=True)


In [ ]:
vib0 = get_normal_mode(numbers, positions, modes[:, 0])
vib1 = get_normal_mode(numbers, positions, modes[:, 1])
vib2 = get_normal_mode(numbers, positions, modes[:, 2])

num_vib0 = get_normal_mode(numbers, positions, nummodes[:, 0])
num_vib1 = get_normal_mode(numbers, positions, nummodes[:, 1])
num_vib2 = get_normal_mode(numbers, positions, nummodes[:, 2])

### Modes

In [ ]:
print(f"This is the bending mode at {vibres.freqs[0]} or {num_vibres.freqs[0]}.")
view1 = p3d.view(width=300, height=300)
view1.addModel(vib0, "xyz", {"vibrate": {"frames": 10, "amplitude": 0.75}})
view1.setViewStyle({"style": "outline", "width": 0.05})
view1.setStyle({"stick": {}, "sphere": {"scale": 0.25}})
view1.animate({"loop": "backAndForth"})
view1.rotate(-90, "x")
view1.zoomTo()

view2 = p3d.view(width=300, height=300)
view2.addModel(num_vib0, "xyz", {"vibrate": {"frames": 10, "amplitude": 0.75}})
view2.setViewStyle({"style": "outline", "width": 0.05})
view2.setStyle({"stick": {}, "sphere": {"scale": 0.25}})
view2.animate({"loop": "backAndForth"})
view2.rotate(-90, "x")
view2.zoomTo()

view1.show()
view2.show()

In [ ]:
print(
    f"This is the symmetric stretching at {vibres.freqs[1]} or {num_vibres.freqs[1]}."
)
view1 = p3d.view(width=300, height=300)
view1.addModel(vib1, "xyz", {"vibrate": {"frames": 10, "amplitude": 0.75}})
view1.setViewStyle({"style": "outline", "width": 0.05})
view1.setStyle({"stick": {}, "sphere": {"scale": 0.25}})
view1.animate({"loop": "backAndForth"})
view1.rotate(-90, "x")
view1.zoomTo()

view2 = p3d.view(width=300, height=300)
view2.addModel(num_vib1, "xyz", {"vibrate": {"frames": 10, "amplitude": 0.75}})
view2.setViewStyle({"style": "outline", "width": 0.05})
view2.setStyle({"stick": {}, "sphere": {"scale": 0.25}})
view2.animate({"loop": "backAndForth"})
view2.rotate(-90, "x")
view2.zoomTo()

view1.show()
view2.show()

In [ ]:
print(
    f"This is the asymmetric stretching mode at {vibres.freqs[2]} or {num_vibres.freqs[2]}."
)
view1 = p3d.view(width=300, height=300)
view1.addModel(vib2, "xyz", {"vibrate": {"frames": 10, "amplitude": 0.75}})
view1.setViewStyle({"style": "outline", "width": 0.05})
view1.setStyle({"stick": {}, "sphere": {"scale": 0.25}})
view1.animate({"loop": "backAndForth"})
view1.rotate(-90, "x")
view1.zoomTo()

view2 = p3d.view(width=300, height=300)
view2.addModel(num_vib2, "xyz", {"vibrate": {"frames": 10, "amplitude": 0.75}})
view2.setViewStyle({"style": "outline", "width": 0.05})
view2.setStyle({"stick": {}, "sphere": {"scale": 0.25}})
view2.animate({"loop": "backAndForth"})
view2.rotate(-90, "x")
view2.zoomTo()

view1.show()
view2.show()